# A machine learning decision tree approach

The iMeta algorithm is essentially a decision tree algorithm, where the variables and threshold for the decisions at each step are manually specified based on human analysis. The simplest way to apply machine learning techniques to the problem would be to use a similar structure to iMeta, which is a decision tree, but use standard ML training techiniques to learn the parameters such as what thresholds to use and how many branches/leaves to have in the tree for the best results. 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import pathlib

In [ ]:
import ipywidgets
import time

In [ ]:
import pandas
import numpy
import matplotlib
import matplotlib.pyplot

In [ ]:
import sklearn
import sklearn.model_selection
import sklearn.preprocessing
import sklearn.tree
import sklearn.metrics

In [ ]:
root_repo_dir = pathlib.Path().absolute().parent
sys.path = [os.path.join(root_repo_dir)] + sys.path

In [ ]:
import xbt.dataset
from xbt.dataset import XbtDataset, UNKNOWN_STR, cat_output_formatter, check_value_found
from xbt.imeta import imeta_classification, XBT_MAX_DEPTH

In [ ]:
# Set up some site specific parameters for the notebook
try:
    environment = os.environ['XBT_ENV_NAME']
except KeyError:
    environment = 'pangeo'

In [ ]:
root_data_dirs = {
    'MO_scitools': '/data/users/shaddad/xbt-data/',
    'pangeo': '/data/misc/xbt-data/',
}
env_date_ranges = {
    'MO_scitools': (1966,2015),
    'pangeo': (1966,2015)
}

In [ ]:
# Set up some dataset specific parameters
root_data_dir = root_data_dirs[environment]
year_range = env_date_ranges[environment]

In [ ]:
experiment_name = 'nb_single_decisionTree_country'
classifier_class = sklearn.tree.DecisionTreeClassifier
classifier_name = 'decision_tree'
suffix='country'

The `classifier_opts` dictionary contains the hyperparameters for the algorithm. For the decision tree, these primarily determine how deep the the tree becomes and how readily leaf nodes are split during the training process. The more splitting and the deeper the tree, the complicated the function in parameter space the tree is creating, but the greater the danger of overfitting.

The criterion is the loss function used to optimise in the training process.

In [ ]:
classifier_opts = {'max_depth': 20,
                   'min_samples_leaf': 1,
                   'criterion': 'gini'
                  }

In [ ]:
cv_metric_names = ['f1_weighted','precision_weighted','recall_weighted']
input_feature_names = ['country','max_depth', 'year']

In [ ]:
input_dir_name = 'csv_with_imeta'
exp_out_dir_name = 'experiment_outputs'

In [ ]:
xbt_input_dir = os.path.join(root_data_dir, input_dir_name)
xbt_output_dir = os.path.join(root_data_dir, exp_out_dir_name, experiment_name)

In [ ]:
# create the output for this experiment if it doesn't exist
if not os.path.isdir(xbt_output_dir):
    os.makedirs(xbt_output_dir)
print(f'outputting to {xbt_output_dir}')

In [ ]:
output_fname_template = 'xbt_output_{classifier}_{suffix}.csv'
result_fname_template = 'xbt_metrics_{classifier}_{suffix}.csv'

## Data preparation

For the purposes of developing and evaluating the machine learning pipeline, we are using the labelled data, that is those profiles where the instrument type is known. Ultimately though it is the unknown data that we want to predict for.

The XbtDataset class has filtered out some bad data including profiles with maximum depths less that 0.0 or greater than 2000.0. There were also some profiles with bad date entries, which have been excluded for now.

Our training and evaluation will all happen within the labelled dataset, but ultimately we want to apply the trained ML algorithms to the unlabelled data, as this is where labels are needed to reduce the uncertainty in the ocean temperature dataset. So we need to design our experiment with this in mind. What we need to know is how well does our chosen algorithm generalise to unseen data. 

So within the labelled dataset we split the data into three groups. Firstly, we set a side some fraction of cruises and all profiles that are from those cruises into a group called "unseen". These profiles will not be used in the training or initial evaluation. These will represent cruises that have no labelled data. 

The remainder of the data will then be split into 2 groups, training and test as is standard for a machine learning pipeline. In splitting the data, we will sample a certain fraction of the profiles for train and the rest will be test. But we don't sample from the whole dataset randomly, as this might result in imbalances in representations of years and instrment types. Rather, for each combination of year and instrument label, sample the required fraction from all profiles matching those values. Doing this all possisble value combination of those 2 parameters should results in all years and instrument type being as well represented as possible in both the test and train sets.

So the algorithm will be fit to the training set. Then we will evaulate performance on the test set. This gives us an idea of how the algorithm will perform on unlabelled profiles where other profiles from the same cruise are labelled. Then we will evaluate performance of the algorithm on the unseen dataset. This suggests how the algorithm will perform on unlabelled profiles where no profiles from that cruise are labelled. Both of these sorts of profiles have been found to be present in the unlabelled data, so it is important to evaluate the algorithm for both cases.

In [ ]:
%%time
xbt_full_dataset = XbtDataset(xbt_input_dir, year_range)

In [ ]:
%%time
xbt_labelled = xbt_full_dataset.filter_obs({'labelled': 'labelled'})

The format of the data in the pandas dataframe is not suitable for feeding into the machine learning algorithms. We have to appropriately encode the data for training. The dataset class has knowledge of the different types of data represented and how each type should be encoded. This happens through the get_ml_dataset method. Calling this with the return_data flag as False doesn't actually encode the data, but initialises the encoder objects required. These encoders are then used for subsequent encoding operations, so encoding are consistent for the whole experiment and results can be compared.

In [ ]:
_ = xbt_labelled.get_ml_dataset(return_data = False)

In [ ]:
_ = xbt_labelled.filter_features(['instrument', 'model', 'manufacturer']).encode_target(return_data = False)

In [ ]:
%%time
unseen_cruise_numbers = xbt_labelled.sample_feature_values('cruise_number', fraction=0.1)

In [ ]:
%%time
xbt_unseen = xbt_labelled.filter_obs({'cruise_number': unseen_cruise_numbers}, mode='include', check_type='in_filter_set')
xbt_working = xbt_labelled.filter_obs({'cruise_number': unseen_cruise_numbers}, mode='exclude', check_type='in_filter_set')

In [ ]:
%%time
imeta_classes = xbt_labelled.xbt_df.apply(imeta_classification, axis=1)

In [ ]:
imeta_model = imeta_classes.apply(lambda t1: t1[0])
imeta_manufacturer = imeta_classes.apply(lambda t1: t1[1])
imeta_instrument = imeta_classes.apply(lambda t1: f'XBT: {t1[0]} ({t1[1]})') 

We are initially also interested in investigating classifier performance looking at prove model and manufacturer separately, as well as looking at the combined model and manufacturer info as a single label and using that as the target feature.

When splitting the data into test and train sets, we want to ensure that there is an even distribution of years and instrument types in train and test sets. In the train/test split function  used here, we divide the data into years and within the year divide it into instrument types, and then randomly sample from each group. This ensure even represenation of years and instruments. There is also a notebook where the splitting is done randomly with no accounting for class imbalances. The results are very similar, suggesting accuracy is not as sensitive to the precise split as it might seem.

In [ ]:
%%time
xbt_train_all, xbt_test_all = xbt_working.train_test_split(refresh=True, features=['instrument', 'year'])

In [ ]:
X_train_all = xbt_train_all.filter_features(input_feature_names).get_ml_dataset()[0]
X_test_all = xbt_test_all.filter_features(input_feature_names).get_ml_dataset()[0]
X_unseen_all = xbt_unseen.filter_features(input_feature_names).get_ml_dataset()[0]
y_model_train_all = xbt_train_all.filter_features(['model']).get_ml_dataset()[0]
y_manuf_train_all = xbt_train_all.filter_features(['manufacturer']).get_ml_dataset()[0]
y_instr_train_all = xbt_train_all.filter_features(['instrument']).get_ml_dataset()[0]
y_model_test_all = xbt_test_all.filter_features(['model']).get_ml_dataset()[0]
y_manuf_test_all = xbt_test_all.filter_features(['manufacturer']).get_ml_dataset()[0]
y_instr_test_all = xbt_test_all.filter_features(['instrument']).get_ml_dataset()[0]
y_model_unseen_all = xbt_unseen.filter_features(['model']).get_ml_dataset()[0]
y_manuf_unseen_all = xbt_unseen.filter_features(['manufacturer']).get_ml_dataset()[0]
y_instr_unseen_all = xbt_unseen.filter_features(['instrument']).get_ml_dataset()[0]

## Training the classifier

We are using the scikit-learn Decision Tree classifier as the closest analogue to the structure of the iMeta algorithm. This tree can have many more nodes and leaves than iMeta though. it is quick to train and evaluate so it is a useful starting point for setting up the ML processing pipelines, as all the scikit-learn classifiers have a common interface. 

For each of the target variables (probe model, probe manufacturer, combined probe model/manufacteer label), we train a Decision Tree classifier, then use it to predict values for the train and test sets. We then calculate the accuracy metrics for each for the whole dataset. 

I am using precision, recall and F1 as fairly standard ML metrics of accuracy. Recall is what has been used in the two previous papers (Palmer et. al, Leahy and Llopis et al) so that is the focus. Support is a useful to see what proportion of the profiles in the dataset belong to each of the different classes.

Documentation
* https://scikit-learn.org/stable/modules/tree.html
* https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html


In [ ]:
metrics_per_class_all = {}
metrics_avg_all = {}

In [ ]:
metrics_per_class_all['model'] = list(xbt_labelled._feature_encoders['model'].classes_)
metrics_per_class_all['manufacturer'] = list(xbt_labelled._feature_encoders['manufacturer'].classes_)
metrics_per_class_all['instrument'] = list(xbt_labelled._feature_encoders['instrument'].classes_)

In [ ]:
%%time
clf_dt_model1 = classifier_class(**classifier_opts)
clf_dt_model1.fit(X_train_all,y_model_train_all)

In [ ]:
y_res_train_model_all = clf_dt_model1.predict(X_train_all)
metrics1 = sklearn.metrics.precision_recall_fscore_support(y_model_train_all, y_res_train_model_all,  labels=list(range(0,len(metrics_per_class_all['model']))) )
metrics_per_class_all.update( {
    'precision_model_train': metrics1[0],
    'recall_model_train': metrics1[1],
    'f1_model_train': metrics1[2],
    'support_model_train': metrics1[3],
})

metrics_avg_all.update({
    'precision_model_train' : sum(metrics1[0] * metrics1[3])/ sum(metrics1[3]),
    'recall_model_train' : sum(metrics1[1] * metrics1[3])/ sum(metrics1[3]),
    'f1_model_train' : sum(metrics1[2] * metrics1[3])/ sum(metrics1[3]),
})

In [ ]:
y_res_test_model = clf_dt_model1.predict(X_test_all)
metrics1 = sklearn.metrics.precision_recall_fscore_support(y_model_test_all, y_res_test_model,  labels=list(range(0,len(metrics_per_class_all['model']))) )
metrics_per_class_all.update( {
    'precision_model_test': metrics1[0],
    'recall_model_test': metrics1[1],
    'f1_model_test': metrics1[2],
    'support_model_test': metrics1[3],
})

metrics_avg_all.update({
    'precision_model_test' : sum(metrics1[0] * metrics1[3])/ sum(metrics1[3]),
    'recall_model_test' : sum(metrics1[1] * metrics1[3])/ sum(metrics1[3]),
    'f1_model_test' : sum(metrics1[2] * metrics1[3])/ sum(metrics1[3]),
})

In [ ]:
y_res_unseen_model = clf_dt_model1.predict(X_unseen_all)
metrics1 = sklearn.metrics.precision_recall_fscore_support(y_model_unseen_all, y_res_unseen_model, labels=list(range(0,len(metrics_per_class_all['model']))) )
metrics_per_class_all.update( {
    'precision_model_unseen': metrics1[0],
    'recall_model_unseen': metrics1[1],
    'f1_model_unseen': metrics1[2],
    'support_model_unseen': metrics1[3],
})

metrics_avg_all.update({
    'precision_model_unseen' : sum(metrics1[0] * metrics1[3])/ sum(metrics1[3]),
    'recall_model_unseen' : sum(metrics1[1] * metrics1[3])/ sum(metrics1[3]),
    'f1_model_unseen' : sum(metrics1[2] * metrics1[3])/ sum(metrics1[3]),
})

In [ ]:
clf_dt_manuf1 = classifier_class(**classifier_opts)
clf_dt_manuf1.fit(X_train_all,y_manuf_train_all)

In [ ]:
y_res_train_manuf_all = clf_dt_manuf1.predict(X_train_all)
metrics1 = sklearn.metrics.precision_recall_fscore_support(y_manuf_train_all, y_res_train_manuf_all, labels=list(range(0,len(metrics_per_class_all['manufacturer']))))
metrics_per_class_all.update( {
    'precision_manuf_train': metrics1[0],
    'recall_manuf_train': metrics1[1],
    'f1_manuf_train': metrics1[2],
    'support_manuf_train': metrics1[3],
})

metrics_avg_all.update({
    'precision_manuf_train' : sum(metrics1[0] * metrics1[3])/ sum(metrics1[3]),
    'recall_manuf_train' : sum(metrics1[1] * metrics1[3])/ sum(metrics1[3]),
    'f1_manuf_train' : sum(metrics1[2] * metrics1[3])/ sum(metrics1[3]),
})



In [ ]:
y_res_test_manuf_all = clf_dt_manuf1.predict(X_test_all)
metrics1 = sklearn.metrics.precision_recall_fscore_support(y_manuf_test_all, y_res_test_manuf_all, labels=list(range(0,len(metrics_per_class_all['manufacturer']))))
metrics_per_class_all.update( {
    'precision_manuf_test': metrics1[0],
    'recall_manuf_test': metrics1[1],
    'f1_manuf_test': metrics1[2],
    'support_manuf_test': metrics1[3],
})
metrics_avg_all.update({
    'precision_manuf_test' : sum(metrics1[0] * metrics1[3])/ sum(metrics1[3]),
    'recall_manuf_test' : sum(metrics1[1] * metrics1[3])/ sum(metrics1[3]),
    'f1_manuf_test' : sum(metrics1[2] * metrics1[3])/ sum(metrics1[3]),
})




In [ ]:
y_res_unseen_manuf_all = clf_dt_manuf1.predict(X_unseen_all)
metrics1 = sklearn.metrics.precision_recall_fscore_support(y_manuf_unseen_all, y_res_unseen_manuf_all, labels=list(range(0,len(metrics_per_class_all['manufacturer']))))
metrics_per_class_all.update( {
    'precision_manuf_unseen': metrics1[0],
    'recall_manuf_unseen': metrics1[1],
    'f1_manuf_unseen': metrics1[2],
    'support_manuf_unseen': metrics1[3],
})
metrics_avg_all.update({
    'precision_manuf_unseen' : sum(metrics1[0] * metrics1[3])/ sum(metrics1[3]),
    'recall_manuf_unseen' : sum(metrics1[1] * metrics1[3])/ sum(metrics1[3]),
    'f1_manuf_unseen' : sum(metrics1[2] * metrics1[3])/ sum(metrics1[3]),
})

In [ ]:
clf_dt_instr1 = classifier_class(**classifier_opts)
clf_dt_instr1.fit(X_train_all,y_instr_train_all)

In [ ]:
%%time
y_res_train_instr_all = clf_dt_instr1.predict(X_train_all)
metrics1 = sklearn.metrics.precision_recall_fscore_support(y_instr_train_all, y_res_train_instr_all, labels=list(range(0,len(metrics_per_class_all['instrument']))) )
metrics_per_class_all.update( {
    'precision_instr_train': metrics1[0],
    'recall_instr_train': metrics1[1],
    'f1_instr_train': metrics1[2],
    'support_instr_train': metrics1[3],
})
metrics_avg_all.update({
    'precision_instr_train' : sum(metrics1[0] * metrics1[3])/ sum(metrics1[3]),
    'recall_instr_train' : sum(metrics1[1] * metrics1[3])/ sum(metrics1[3]),
    'f1_instr_train' : sum(metrics1[2] * metrics1[3])/ sum(metrics1[3]),
})

In [ ]:
%%time
y_res_test_instr_all = clf_dt_instr1.predict(X_test_all)
metrics1 = sklearn.metrics.precision_recall_fscore_support(y_instr_test_all, y_res_test_instr_all, labels=list(range(0,len(metrics_per_class_all['instrument']))) )
metrics_per_class_all.update( {
    'precision_instr_test': metrics1[0],
    'recall_instr_test': metrics1[1],
    'f1_instr_test': metrics1[2],
    'support_instr_test': metrics1[3],
})
metrics_avg_all.update({
    'precision_instr_test' : sum(metrics1[0] * metrics1[3])/ sum(metrics1[3]),
    'recall_instr_test' : sum(metrics1[1] * metrics1[3])/ sum(metrics1[3]),
    'f1_instr_test' : sum(metrics1[2] * metrics1[3])/ sum(metrics1[3]),
})

In [ ]:
%%time
y_res_unseen_instr_all = clf_dt_instr1.predict(X_unseen_all)
metrics1 = sklearn.metrics.precision_recall_fscore_support(y_instr_unseen_all, y_res_unseen_instr_all, labels=list(range(0,len(metrics_per_class_all['instrument']))) )
metrics_per_class_all.update( {
    'precision_instr_unseen': metrics1[0],
    'recall_instr_unseen': metrics1[1],
    'f1_instr_unseen': metrics1[2],
    'support_instr_unseen': metrics1[3],
})
metrics_avg_all.update({
    'precision_instr_unseen' : sum(metrics1[0] * metrics1[3])/ sum(metrics1[3]),
    'recall_instr_unseen' : sum(metrics1[1] * metrics1[3])/ sum(metrics1[3]),
    'f1_instr_unseen' : sum(metrics1[2] * metrics1[3])/ sum(metrics1[3]),
})

In [ ]:
df_metrics_per_class_model = pandas.DataFrame.from_dict({k1:v1 for k1,v1 in metrics_per_class_all.items() if 'model' in k1})
df_metrics_per_class_manuf = pandas.DataFrame.from_dict({k1:v1 for k1,v1 in metrics_per_class_all.items() if 'manuf' in k1})
df_metrics_per_class_instr = pandas.DataFrame.from_dict({k1:v1 for k1,v1 in metrics_per_class_all.items() if 'instr' in k1})

In [ ]:
df_metrics_avg = pandas.DataFrame.from_dict({
    'target': ['model_train','model_test', 'model_unseen', 'manufacturer_train','manufacturer_test', 'manufacturer_unseen','instrument_train','instrument_test', 'instrument_unseen'],
    'precision': [v1 for k1,v1 in metrics_avg_all.items() if 'precision' in k1],
    'recall': [v1 for k1,v1 in metrics_avg_all.items() if 'recall' in k1],
    'f1': [v1 for k1,v1 in metrics_avg_all.items() if 'f1' in k1],
})

In [ ]:
df_metrics_avg

# Classification result plots

The plots below show the results for the whole XBT dataset. We see that the DT classifier performs almost perfectly on the training data, but performance degrades for the test data, and further still for the profiles from unseen cruises. So it does not seem to generalise that well, but results are still better than other algorithms where the train and test results are closer together. Performance is poor for classes with very little support in the training dataset, as one might expect.

In [ ]:
df_metrics_per_class_model

In [ ]:
fig_results_all_dt = matplotlib.pyplot.figure('xbt_results_all_dt', figsize=(24,24))
axis_model_metrics = fig_results_all_dt.add_subplot(321)
_ = df_metrics_per_class_model.plot.bar(x='model',y=['recall_model_train', 'recall_model_test', 'recall_model_unseen'], ax=axis_model_metrics)
axis_model_support = fig_results_all_dt.add_subplot(322)
_ = df_metrics_per_class_model.plot.bar(x='model',y=['support_model_train', 'support_model_test', 'support_model_unseen'], ax=axis_model_support)
axis_manuf_metrics = fig_results_all_dt.add_subplot(323)
_ = df_metrics_per_class_manuf.plot.bar(x='manufacturer', y=['recall_manuf_train','recall_manuf_test','recall_manuf_unseen'],ax=axis_manuf_metrics)
axis_manuf_support = fig_results_all_dt.add_subplot(324)
_ = df_metrics_per_class_manuf.plot.bar(x='manufacturer',y=['support_manuf_train', 'support_manuf_test', 'support_manuf_unseen'], ax=axis_manuf_support)
axis_instr_metrics = fig_results_all_dt.add_subplot(325)
_ = df_metrics_per_class_instr.plot.bar(x='instrument', y=['recall_instr_train','recall_instr_test','recall_instr_unseen'],ax=axis_instr_metrics)
axis_instr_support = fig_results_all_dt.add_subplot(326)
_ = df_metrics_per_class_instr.plot.bar(x='instrument',y=['support_instr_train', 'support_instr_test', 'support_instr_unseen'], ax=axis_instr_support)


The plot of performance per class show that results generalise best where there are a substantial number of profiles of that probe type in the training and test sets. Further work may be needed to consider how to boost the performance of underepresented classes. Although these may be only a small number of profiles, we should also consider whether the measurements they contribute are in under represented areas and thus may have a disproportionately alrge impact on the final teprature values in some parts of the world in some years.

In [ ]:
df_metrics_avg.plot.bar(figsize=(18,12), x='target', y='recall')

## Classification results

The contents of the XBT dataset varies over the time period, so previous papers have looked at classification accuracy (recall) year by year to evaluate how performance varies with different distribution of probe types.

To do this we apply the classifier to the train, test  and unseen profile groups for each year separately. For each year we calculate the target metrics for each taret variable and plot the results. We compare the results to those achieved by the iMeta algorithm.

In [ ]:
def score_year(xbt_df, year, clf, input_features, target_feature):
    X_year = xbt_df.filter_obs({'year': year}, ).filter_features(input_features).get_ml_dataset()[0]
    y_year = xbt_df.filter_obs({'year': year} ).filter_features([target_feature]).get_ml_dataset()[0]
    y_res_year = clf.predict(X_year)
    metric_year = sklearn.metrics.precision_recall_fscore_support(
        y_year, y_res_year, average='micro')
    return metric_year

In [ ]:
eval_progress = ipywidgets.IntProgress(min=env_date_ranges[environment][0],
                                           max= env_date_ranges[environment][1],
                                          description='Evaluating',
                                          bar_style='info')
eval_progress

In [ ]:
results_by_year = {}

In [ ]:
%%time
for year in range(env_date_ranges[environment][0],env_date_ranges[environment][1]):
    results_by_year[year] = {
        'metric_train_model' : score_year(xbt_train_all, year, clf_dt_model1, input_feature_names, 'model'),
        'metric_test_model' : score_year(xbt_test_all, year, clf_dt_model1, input_feature_names, 'model'),
        'metric_unseen_model' : score_year(xbt_unseen, year, clf_dt_model1, input_feature_names, 'model'),
        'metric_train_manuf' : score_year(xbt_train_all, year, clf_dt_manuf1, input_feature_names, 'manufacturer'),
        'metric_test_manuf' : score_year(xbt_test_all, year, clf_dt_manuf1, input_feature_names, 'manufacturer'),
        'metric_unseen_manuf' : score_year(xbt_unseen, year, clf_dt_manuf1, input_feature_names, 'manufacturer'),
        'metric_train_instr' : score_year(xbt_train_all, year, clf_dt_instr1, input_feature_names, 'instrument'),
        'metric_test_instr' : score_year(xbt_test_all, year, clf_dt_instr1, input_feature_names, 'instrument'),
        'metric_unseen_instr' : score_year(xbt_unseen, year, clf_dt_instr1, input_feature_names, 'instrument'),
    }
    eval_progress.value = year


In [ ]:
recall_by_year = pandas.DataFrame.from_dict({ 
    'year':  list(results_by_year.keys()),
    'recall_train_model' : [m1['metric_train_model'][1] for y1,m1 in results_by_year.items()],
    'recall_test_model' : [m1['metric_test_model'][1] for y1,m1 in results_by_year.items()],
    'recall_unseen_model' : [m1['metric_unseen_model'][1] for y1,m1 in results_by_year.items()],
    'recall_train_manuf' : [m1['metric_train_manuf'][1] for y1,m1 in results_by_year.items()],
    'recall_test_manuf' : [m1['metric_test_manuf'][1] for y1,m1 in results_by_year.items()],
    'recall_unseen_manuf' : [m1['metric_unseen_manuf'][1] for y1,m1 in results_by_year.items()],
    'recall_train_instr' : [m1['metric_train_instr'][1] for y1,m1 in results_by_year.items()],
    'recall_test_instr' : [m1['metric_test_instr'][1] for y1,m1 in results_by_year.items()],
    'recall_unseen_instr' : [m1['metric_unseen_instr'][1] for y1,m1 in results_by_year.items()],
})


In [ ]:
model_encoder = xbt_labelled._feature_encoders['model']
manuf_encoder = xbt_labelled._feature_encoders['manufacturer']
instr_encoder = xbt_labelled._feature_encoders['instrument']

In [ ]:
%%time
eval_progress.value = env_date_ranges[environment][0]
imeta_results = []
for year in range(env_date_ranges[environment][0],env_date_ranges[environment][1]):
    y_imeta_model = model_encoder.transform(pandas.DataFrame(imeta_model[xbt_labelled.xbt_df.year == year]))
    xbt_model1 = model_encoder.transform(pandas.DataFrame(xbt_labelled.xbt_df[xbt_labelled.xbt_df.year == year].model))

    y_imeta_manuf = manuf_encoder.transform(pandas.DataFrame(imeta_manufacturer[xbt_labelled.xbt_df.year == year]))
    xbt_manufacturer1 = manuf_encoder.transform(pandas.DataFrame(xbt_labelled.xbt_df[xbt_labelled.xbt_df.year == year].manufacturer))

    y_imeta_instr = instr_encoder.transform(pandas.DataFrame(imeta_instrument[xbt_labelled.xbt_df.year == year]))
    xbt_instr1 = instr_encoder.transform(pandas.DataFrame(xbt_labelled.xbt_df[xbt_labelled.xbt_df.year == year].instrument))
    
    
    (im_pr_model, im_rec_model, im_f1_model, im_sup_model) = sklearn.metrics.precision_recall_fscore_support(xbt_model1, y_imeta_model,average='micro')
    (im_pr_manuf, im_rec_manuf, im_f1_manuf, im_sup_manuf) = sklearn.metrics.precision_recall_fscore_support(xbt_manufacturer1, y_imeta_manuf,average='micro')
    (im_pr_instr, im_rec_instr, im_f1_instr, im_sup_instr) = sklearn.metrics.precision_recall_fscore_support(xbt_instr1, y_imeta_instr,average='micro')

    imeta_results += [{'year': year,
                       'imeta_model_recall': im_rec_model,
                       'imeta_model_precision': im_pr_model,
                       'imeta_manuf_recall': im_rec_manuf,
                       'imeta_manuf_precision': im_pr_manuf,
                       'imeta_instr_recall': im_rec_instr,
                       'imeta_instr_precision': im_pr_instr,
                      }]
    eval_progress.value = year

In [ ]:
imeta_res_df = pandas.DataFrame.from_records(imeta_results)

In [ ]:
results_df = pandas.merge(recall_by_year, imeta_res_df).merge(
    pandas.DataFrame.from_dict({
        'year': xbt_labelled['year'].value_counts(sort=False).index,
        'num_samples': xbt_labelled['year'].value_counts(sort=False).values,
    }))

In [ ]:
fig_model_recall_results = matplotlib.pyplot.figure('xbt_model_recall', figsize=(18,6))
ax_model_recall_results = fig_model_recall_results.add_subplot(131, title='XBT model recall results')
_ = results_df.plot.line(x='year',y=['recall_train_model','recall_test_model', 'recall_unseen_model', 'imeta_model_recall'], ax=ax_model_recall_results)
ax_manuf_recall_results = fig_model_recall_results.add_subplot(132, title='XBT manufacturer recall results')
_ = results_df.plot.line(x='year',y=['recall_train_manuf','recall_test_manuf', 'recall_unseen_manuf', 'imeta_manuf_recall'], ax=ax_manuf_recall_results)
ax_instr_recall_results = fig_model_recall_results.add_subplot(133, title='XBT instrument recall results')
_ = results_df.plot.line(x='year',y=['recall_train_instr','recall_test_instr', 'recall_unseen_instr', 'imeta_instr_recall'], ax=ax_instr_recall_results)

We see that for both the test and unseen profiles, the decision tree performs at least as well as the iMeta algorithm, and substantially better for some years, particularly the 1985-1995 era when there are a lot of profiles and a lot of them are missing probe type meta data. We can see how the algorithm genalises better in some years that others. The performance on the unseen profiles is slightly worse than the test profiles, but roughly comparable except in a few years where performance is substantially worse. This merits further investigation.

In [ ]:
results_df['improvement_model'] = results_df.apply(lambda r1: ((r1['recall_test_model'] /  r1['imeta_model_recall'])-1)*100.0 , axis=1)
results_df['improvement_manuf'] = results_df.apply(lambda r1: ((r1['recall_test_manuf'] /  r1['imeta_manuf_recall'])-1)*100.0 , axis=1)
results_df['improvement_instr'] = results_df.apply(lambda r1: ((r1['recall_test_instr'] /  r1['imeta_instr_recall'])-1)*100.0 , axis=1)

In [ ]:
fig_num_samples_per_year = matplotlib.pyplot.figure('fig_num_samples_per_year', figsize=(16,16))
ax_num_samples = fig_num_samples_per_year.add_subplot(221, title='number of samples per year')
_ = results_df.plot.line(ax=ax_num_samples, x='year',y=['num_samples'],c='purple' )
ax_num_samples = fig_num_samples_per_year.add_subplot(222, title='improvement model per year')
_ = results_df.plot.line(ax=ax_num_samples, x='year',y=['improvement_model'], c='green' )
ax_num_samples = fig_num_samples_per_year.add_subplot(223, title='improvement manufacturer per year')
_ = results_df.plot.line(ax=ax_num_samples, x='year',y=['improvement_manuf'], c='green' )
ax_num_samples = fig_num_samples_per_year.add_subplot(224, title='improvement instrument per year')
_ = results_df.plot.line(ax=ax_num_samples, x='year',y=['improvement_instr'], c='green' )


We see substantial improvements in recall some years, particularly the early 1990s. This is a postive outcome as this where the high percent of unlabelled data is found.

In [ ]:
results_df.to_csv(os.path.join(xbt_output_dir, result_fname_template.format(classifier=classifier_name,
                                                                            suffix=suffix)))

### Outputing the data

The output format of the probe classifications is both as a string label, as as a one hot encoding (OHE). OHE econdes the results as as nxm array with each of n rows representing a profile and each of m columns a profile type, with zeroes in most entries except a 1 in the column of the probe type for that profile. This is used as it can be genealised to providing probabilities, where instead of just ones and zeroes, each column represents the probability of the profile being from a particular probe type, with values between 0 and 1 that sum to 1 for each row. This will be explored further in subsequent notebooks.



In [ ]:
# checker functions check each element of the profile metadata that could be a problem. The checkers are constructed from the labelled data subset.
checkers_labelled = {f1: c1 for f1, c1 in xbt_labelled.get_checkers().items() if f1 in input_feature_names}

In [ ]:
result_feature_name = 'instrument_res_dt'

To filter based on what profiles we can use for predicting, we need some way of checking each profile we create these checkers from the labelled dataset, because the subset of data that was used for training determines what subset is valid for prediction. For example, if a country is not present in the training data, then the prediction function won't be able to handle that profile to predict a probe model and manufacturer. Profiles that are not handled by the trained classifier will get the label "UNKNOWN".


In [ ]:
%%time
xbt_predictable = xbt_full_dataset.filter_predictable(checkers_labelled)

In [ ]:
res_ml1 = clf_dt_instr1.predict(xbt_predictable.filter_features(input_feature_names).get_ml_dataset()[0])
res2 = list(xbt_labelled._feature_encoders['instrument'].inverse_transform(res_ml1).reshape(-1))

In [ ]:
xbt_predictable.xbt_df[result_feature_name] = res2

In [ ]:
xbt_full_dataset.merge_features(xbt_predictable, [result_feature_name],
                               fill_values = {result_feature_name: UNKNOWN_STR},
                               feature_encoders={result_feature_name: xbt_labelled._feature_encoders['instrument']},
                                target_encoders={result_feature_name: xbt_labelled._target_encoders['instrument']},
                               output_formatters={result_feature_name: [cat_output_formatter]})

In [ ]:
xbt_full_dataset.output_data(out_dir=xbt_output_dir,
                             fname_template=output_fname_template,
                             exp_name=experiment_name,
                             output_split=xbt.common.OUTPUT_SINGLE,
                             target_features=[result_feature_name])

## Summary
We can see so far that the basic tree xbt_output_dirch seems to be outperforming iMeta. Further work needs to be done to calculate these results more rigorously  using cross validation. 

This notebooks shows the basic classification pipeline. In the following notebooks, this will be used in the following ways
* trying other machine learning algorithms
* trying other input combinations
* do hyper parameter tuning to find the best hyperparameters
* cross-validation to check that the results are not anomolously good or bad based on the particular train/test split
* run through all the different combinations and compare recall values to choose a best algorithm and input feature set.

In addition, this pipeline will ultimately need to go into a script to do the training to be used by the ocean scientists to produce classifications that feed into the ocean datasets.
